In [1]:
#Welcome! This is my first attempt at constructing a DNN
#just run every cell! everything is completely organized.
#however, this model predicts the price at a monthly rate, so it's not exactly the best for a six-month long prediction
#but the monthly results are great! rmse fluctuate somewhere between 3-4, which is good enough

#code is runnable

In [2]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import pandas as pd
 
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import  train_test_split
 
import tensorflow as tf
from tensorflow.keras import regularizers
import pandas as pd

In [3]:
data = pd.read_csv('crude oil dataset/Cushing_OK_WTI_Spot_Price_FOB.csv',parse_dates=['Date'])
data = data.reindex(index=data.index[::-1])
split_time = 32
series = np.array(data.Price)

#time_train = time[:split_time]
x_train = series[:split_time]

#time_valid = time[split_time:]
x_valid = series[split_time:]


In [4]:
window_size = 2
batch_size = 32
shuffle_buffer_size = 1000

In [5]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    """Generates dataset windows

    Args:
      series (array of float) - contains the values of the time series
      window_size (int) - the number of time steps to average
      batch_size (int) - the batch size
      shuffle_buffer(int) - buffer size to use for the shuffle method

    Returns:
      dataset (TF Dataset) - TF Dataset containing time windows
    """
  
    # Generate a TF Dataset from the series values
    dataset = tf.data.Dataset.from_tensor_slices(series)
    
    # Window the data but only take those with the specified size
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
    
    # Flatten the windows by putting its elements in a single batch
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))

    # Create tuples with features and labels 
    dataset = dataset.map(lambda window: (window[:-1], window[-1]))

    # Shuffle the windows
    dataset = dataset.shuffle(shuffle_buffer)
    
    # Create batches of windows
    dataset = dataset.batch(batch_size).prefetch(1)
    
    return dataset

In [6]:
dataset = windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)

In [7]:
# Build the model
model_baseline = tf.keras.models.Sequential([
    tf.keras.layers.Dense(10, input_shape=[window_size], activation="relu"), 
    tf.keras.layers.Dense(10, activation="relu"), 
    tf.keras.layers.Dense(1)
])

# Print the model summary
model_baseline.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                30        
                                                                 
 dense_1 (Dense)             (None, 10)                110       
                                                                 
 dense_2 (Dense)             (None, 1)                 11        
                                                                 
Total params: 151
Trainable params: 151
Non-trainable params: 0
_________________________________________________________________


In [8]:
# Set the training parameters
model_baseline.compile(loss="mse", optimizer=tf.keras.optimizers.SGD(learning_rate=1e-6, momentum=0.9))

In [9]:
model_baseline.fit(dataset,epochs=200)

Epoch 1/200
1/1 [==============================] - 2s 2s/step - loss: 329.0299
Epoch 2/200
1/1 [==============================] - 0s 12ms/step - loss: 327.6282
Epoch 3/200
1/1 [==============================] - 0s 13ms/step - loss: 324.9775
Epoch 4/200
1/1 [==============================] - 0s 13ms/step - loss: 321.2256
Epoch 5/200
1/1 [==============================] - 0s 12ms/step - loss: 316.5131
Epoch 6/200
1/1 [==============================] - 0s 13ms/step - loss: 310.9727
Epoch 7/200
1/1 [==============================] - 0s 13ms/step - loss: 304.7280
Epoch 8/200
1/1 [==============================] - 0s 13ms/step - loss: 297.8935
Epoch 9/200
1/1 [==============================] - 0s 13ms/step - loss: 290.5737
Epoch 10/200
1/1 [==============================] - 0s 13ms/step - loss: 282.8638
Epoch 11/200
1/1 [==============================] - 0s 12ms/step - loss: 274.8497
Epoch 12/200
1/1 [==============================] - 0s 12ms/step - loss: 266.6082
Epoch 13/200
1/1 [=========

1/1 [==============================] - 0s 12ms/step - loss: 19.9544
Epoch 102/200
1/1 [==============================] - 0s 12ms/step - loss: 19.2475
Epoch 103/200
1/1 [==============================] - 0s 12ms/step - loss: 18.5734
Epoch 104/200
1/1 [==============================] - 0s 11ms/step - loss: 17.9235
Epoch 105/200
1/1 [==============================] - 0s 12ms/step - loss: 17.3019
Epoch 106/200
1/1 [==============================] - 0s 11ms/step - loss: 16.7029
Epoch 107/200
1/1 [==============================] - 0s 12ms/step - loss: 16.1261
Epoch 108/200
1/1 [==============================] - 0s 12ms/step - loss: 15.5695
Epoch 109/200
1/1 [==============================] - 0s 13ms/step - loss: 15.0325
Epoch 110/200
1/1 [==============================] - 0s 12ms/step - loss: 14.5153
Epoch 111/200
1/1 [==============================] - 0s 12ms/step - loss: 14.0179
Epoch 112/200
1/1 [==============================] - 0s 11ms/step - loss: 13.5391
Epoch 113/200
1/1 [===========

In [10]:
def plot_series(time, series, format="-", start=0, end=None):
    """
    Visualizes time series data

    Args:
      time (array of int) - contains the time steps
      series (array of int) - contains the measurements for each time step
      format - line style when plotting the graph
      label - tag for the line
      start - first time step to plot
      end - last time step to plot
    """

    # Setup dimensions of the graph figure
    plt.figure(figsize=(10, 6))
    
    if type(series) is tuple:

        for series_num in series:
        # Plot the time series data
            plt.plot(time[start:end], series_num[start:end], format)

    else:
      # Plot the time series data
        plt.plot(time[start:end], series[start:end], format)

    # Label the x-axis
    plt.xlabel("Time")

    # Label the y-axis
    plt.ylabel("Value")

    # Overlay a grid on the graph
    plt.grid(True)

    # Draw the graph on screen
    plt.show()

In [11]:
# Initialize a list
forecast = []

# Reduce the original series
forecast_series = series[split_time - window_size:]

# Use the model to predict data points per window size
for time in range(len(forecast_series) - window_size):
  forecast.append(model_baseline.predict(forecast_series[time:time + window_size][np.newaxis]))

# Convert to a numpy array and drop single dimensional axes
results = np.array(forecast).squeeze()

In [12]:
# Compute the metrics
rmse = tf.keras.metrics.mean_squared_error(x_valid, results).numpy()
rmse = np.sqrt(rmse)
print(rmse)
print(tf.keras.metrics.mean_absolute_error(x_valid, results).numpy())

6.413183
4.2751613
